This script enables one to take an existing gexf file, and extract a list of BodyIDs involved in particular subnetwork motifs. A new motif can be added to the existing group. 

Inputs
Graph: a gexf file that includes node labels (with their accompanying attributes), and edge information
LPTCs_Right/Left: for this particular project, we are trying to detect particular motifs between right and left hemisphere LPTCs (Lobula Plate Tangentional Cells), hence, inputting them as right and left hemisphere groups
Path_N: the minimum allowed path length between nodes (Body IDs)
Min: the minimum connection weight between nodes

Output
Confirmed Paths: A list of confirmed motif paths, showing both the type of motif (as indicated in the script), as well as those BodyIDs involved

In [1]:



def k_shortest_paths(G, source, target, k, weight=None):
    import networkx as nx
    from itertools import islice
    return list(islice(nx.shortest_simple_paths(G, source, target, weight=weight), k))


#find paths between Right and Left Hemisphere LPTCs
def paths_between_LPTCS(Graph,LPTCs_Right,LPTCs_Left,Path_N,Min):
    
    import networkx as nx
    
    G_M = nx.read_gexf(Graph)
    G_D = nx.Graph(G_M)
    
    Paths = []
    
    #find N number of shortest paths between Right and Left hemisphere LPTCs
    for ID1 in LPTCs_Right:
        for ID2 in LPTCs_Left:
            try:
                for path in k_shortest_paths(G_D, str(ID1), str(ID2), Path_N):
                    if len(path) >= 3:
                        Paths.append(path)
            except:
                pass
        
    
    Confirmed_Paths = []   
    
    #Motif 1a ==== LPTCR ---> T1 ---> T3 ---> T2 ---> LPTCL
    for P in Paths:
        l = len(P)
        count = 0
        for i in range(0,l-2,1):
            if G_M.has_edge(str(P[i]), str(P[i+1])) == True:
                count += 1
         
        if count == l-1:
            Confirmed_Paths.append(["Motif1a",P])
            
    #Motif 1b ==== LPTCR <--- T1 <--- T3 <--- T2 <--- LPTCL
    for P in Paths:
        l = len(P)
        count = 0
        for i in range(l-1,1,1):
            if G_M.has_edge(str(P[i]), str(P[i-1])) == True:
                count += 1
         
        if count == l-1:
            Confirmed_Paths.append(["Motif1b",P])
                        
    
      
    #Motif 2 ==== LPTC1 ---> T1 ---> T3 <--- T2 <--- LPTC2
    for P in Paths:
        l = len(P)
        
        if l != 5:
            continue
        
        #check that T1 is output of LPTCR
        if G_M.has_edge(str(P[0]), str(P[1])) == False:
            continue
           
        if G_M.number_of_edges(str(P[0]),str(P[1])) < Min:
            continue
          
        #check that T3 is output of T1
        if G_M.has_edge(str(P[1]), str(P[2])) == False:
            continue
            
        if G_M.number_of_edges(str(P[1]),str(P[2])) < Min:
            continue
        
        #check that T2 is output of LPTCL
        if G_M.has_edge(str(P[4]), str(P[3])) == False:
            continue
            
        if G_M.number_of_edges(str(P[4]),str(P[3])) < Min:
            continue
        
        #check that T3 is output of T2
        if G_M.has_edge(str(P[3]), str(P[2])) == False:
            continue
            
        if G_M.number_of_edges(str(P[3]),str(P[2])) < Min:
            continue
              
            
        Confirmed_Paths.append(["Motif2",P])
        
        
    #Motif 3 ==== LPTCR ---> T1 <--- LPTCL 
    for P in Paths:
        l = len(P)
        
        if l != 3:
            continue
        
        #check that T1 is output of LPTCR
        if G_M.has_edge(str(P[0]), str(P[1])) == False:
            continue
            
        if G_M.number_of_edges(str(P[0]),str(P[1])) < Min:
            continue
          
        #check that T1 is output of LPTCL
        if G_M.has_edge(str(P[2]), str(P[1])) == False:
            continue
            
        if G_M.number_of_edges(str(P[2]),str(P[1])) < Min:
            continue
        
            
        Confirmed_Paths.append(["Motif3",P])
    
    
    f = open("Motifs.txt", "w")
    for path in Confirmed_Paths:
        f.write(str(path)+ '\n')
   
    

In [2]:
def subgraph(Graph, Node_List):
    import networkx as nx
    
    List= []
    for item in Node_List:
        List.append(str(item))
        
    
    G = nx.read_gexf(Graph)
    H = G.subgraph(List)
    
    #write gefx file for use in Gephi
    nx.write_gexf(H, 'Subgraph'+'.gexf')  